In [ ]:
import yfinance as yf
from backtesting import Backtest
import datetime as dt

import sys
import os
sys.path.append(os.path.abspath("/home/da2343/robust-algo-trader/strategies"))
from strategies import *


In [ ]:
import pandas as pd
import numpy as np
from backtesting import Backtest
from backtesting import Strategy
# from backtesting.test import EURUSD
import talib
import sys
import os
# sys.path.append(os.path.abspath("/home/da2343/robust-algo-trader/strategies"))
# from strategies import CustomStrategy



np.random.seed(1)

EURUSD = pd.read_table('/home/da2343/robust-algo-trader/data/EURUSD/EURUSD_H1_200702210000_202304242100.tsv')
# remove the following columns <TICKVOL>, <VOL> and <SPREAD>
EURUSD = EURUSD.drop(['<TICKVOL>', '<VOL>', '<SPREAD>'], axis=1)
# rename the columns
EURUSD = EURUSD.rename(columns={'<DATE>': 'Date', 
                                '<TIME>': 'Time', 
                                '<OPEN>': 'Open', 
                                '<HIGH>': 'High', 
                                '<LOW>': 'Low', 
                                '<CLOSE>': 'Close'})
# combine the date and time columns
EURUSD['Date_Time'] = EURUSD['Date'] + ' ' + EURUSD['Time']
# remove the date and time columns
EURUSD = EURUSD.drop(['Date', 'Time'], axis=1)
# convert the date_time column to datetime
EURUSD['Date_Time'] = pd.to_datetime(EURUSD['Date_Time'], format='%Y%m%d %H:%M:%S.%f')
# Set index of EURUSD
# EURUSD.index = EURUSD['Date_Time']
# EURUSD.set_index('Date_Time')

# generate signals and store the signal in a new column called 'Signal'
# 1 for buy and -1 for sell and  0 for close
# generate random signals
# EURUSD['Signal'] = np.random.randint(-1, 2, size=len(EURUSD))

from backtesting import Backtest
from backtesting import Strategy

class CustomStrategy(Strategy):
    def init(self):
        pass
    
    def next(self):
        current_signal = self.data.Signal[-1]
        if current_signal == 1:
            self.buy()
        elif current_signal == -2:
            self.position.close() 
            pass       
        


# lambda function to generate signals
EURUSD['Signal'] = 0

# # generate signals based on the Hammer candlestick pattern, using talib
EURUSD['CDLHAMMER'] = talib.CDLHIKKAKEMOD(EURUSD['Open'], EURUSD['High'], EURUSD['Low'], EURUSD['Close'])

for i in range(len(EURUSD)):
    x = EURUSD.loc[i]
   # check if any of the 4 previous rows have a signal
    if i > 0:
      x_1 = EURUSD.loc[i - 1]
      if x_1['Signal'] == 1:
        continue
      
    if i > 1:  
      x_2 = EURUSD.loc[i - 2]
      if x_2['Signal'] == 1:
        continue
      
    if i > 2:  
      x_3 = EURUSD.loc[i - 3]
      if x_3['Signal'] == 1:
        continue
    
    if x['CDLHAMMER'] == 100:
        EURUSD.loc[i, "Signal"] = 1
        EURUSD.loc[i+1, "Signal"] = 0
        EURUSD.loc[i+2, "Signal"] = 0
        EURUSD.loc[i+3, "Signal"] = -2

EURUSD.index = EURUSD['Date_Time']
bt = Backtest(EURUSD, CustomStrategy, commission=.002,
              cash=2000,
              exclusive_orders=True)
stats = bt.run()

bt.plot ()
print(stats)
stats._strategy
bt.plot(plot_volume=False, plot_pl=False)
stats.tail()
print(stats['_trades'])               


In [ ]:

# print(EURUSD)

mccv = MonteCarloCV(n_splits=5, 
                    train_size=0.6, 
                    test_size=0.1, 
                    gap=0)

for train_index, test_index in mccv.split(EURUSD):
    # print("TRAIN:", train_index, "TEST:", test_index)
    train_set = EURUSD.loc[train_index]
    # set the date_time column as index
    train_set = train_set.set_index('Date_Time')
    bt = Backtest(train_set, SmaCross, commission=.002,
              cash=10_000,
              exclusive_orders=True)
    bt.run()
    stats = bt.optimize(n1=range(5, 50, 1),
                    n2=range(10, 100, 1),
                    maximize='Win Rate [%]',
                    max_tries=100,
                    random_state=0,
                    constraint=lambda param: param.n1 < param.n2)
    bt.plot()
    
    print(stats)
    break
    
    # X_train, X_test = EURUSD[train_index], EURUSD[test_index]
    # y_train, y_test = EURUSD[train_index], EURUSD[test_index]

In [ ]:

# symbol = 'AAPL'
symbol = 'GOOG'

# Define the start and end dates for the data
# start_date = '2022-01-11'
# end_date = '2023-01-01'

# set current date as end date
end_date = dt.datetime.now().strftime('%Y-%m-%d')

# Download the data using yfinance
data = yf.download(symbol, start="2021-01-01", end=end_date, interval='1d')


bt = Backtest(data, SmaCross, commission=.002,
              cash=2_000,
              exclusive_orders=True)
bt.run()

stats = bt.optimize(n1=range(5, 50, 1),
                    n2=range(10, 100, 1),
                    maximize='Win Rate [%]',
                    constraint=lambda param: param.n1 < param.n2)

bt.plot()
stats
stats._strategy
bt.plot(plot_volume=False, plot_pl=False)
stats.tail()
stats['_trades']


In [ ]:
from typing import List, Generator
import numpy as np
from sklearn.model_selection._split import _BaseKFold
from sklearn.utils.validation import indexable, _num_samples


class MonteCarloCV(_BaseKFold):

    def __init__(self,
                 n_splits: int,
                 train_size: float,
                 test_size: float,
                 gap: int = 0):
        """
        Monte Carlo Cross-Validation

        Holdout applied in multiple testing periods
        Testing origin (time-step where testing begins) is randomly chosen according to a monte carlo simulation

        :param n_splits: (int) Number of monte carlo repetitions in the procedure
        :param train_size: (float) Train size, in terms of ratio of the total length of the series
        :param test_size: (float) Test size, in terms of ratio of the total length of the series
        :param gap: (int) Number of samples to exclude from the end of each train set before the test set.
        """

        self.n_splits = n_splits
        self.n_samples = -1
        self.gap = gap
        self.train_size = train_size
        self.test_size = test_size
        self.train_n_samples = 0
        self.test_n_samples = 0

        self.mc_origins = []

    def split(self, X, y=None, groups=None) -> Generator:
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """

        X, y, groups = indexable(X, y, groups)
        self.n_samples = _num_samples(X)

        self.train_n_samples = int(self.n_samples * self.train_size) - 1
        self.test_n_samples = int(self.n_samples * self.test_size) - 1

        # Make sure we have enough samples for the given split parameters
        if self.n_splits > self.n_samples:
            raise ValueError(
                f'Cannot have number of folds={self.n_splits} greater'
                f' than the number of samples={self.n_samples}.'
            )
        if self.train_n_samples - self.gap <= 0:
            raise ValueError(
                f'The gap={self.gap} is too big for number of training samples'
                f'={self.train_n_samples} with testing samples={self.test_n_samples} and gap={self.gap}.'
            )

        indices = np.arange(self.n_samples)

        selection_range = np.arange(self.train_n_samples + 1, self.n_samples - self.test_n_samples - 1)

        self.mc_origins = \
            np.random.choice(a=selection_range,
                             size=self.n_splits,
                             replace=True)

        for origin in self.mc_origins:
            if self.gap > 0:
                train_end = origin - self.gap + 1
            else:
                train_end = origin - self.gap
            train_start = origin - self.train_n_samples - 1

            test_end = origin + self.test_n_samples

            yield (
                indices[train_start:train_end],
                indices[origin:test_end],
            )

    def get_origins(self) -> List[int]:
        return self.mc_origins